In [1]:
'''Loading libraries'''
import pandas as pd 
import numpy as np 
from scipy.stats import gamma, dirichlet, beta, nbinom, norm


'''
Notes: 
1) Likelihood_F I had to add a constant to avoid log(0). Check for a better solution
'''


'''Important parameters I need to constantly change'''
k = 10


In [2]:
'''Loading dataset'''
#filename = "C:\\Users\\raoki\\Documents\\GitHub\\project_spring2019\\Data\\data_final.csv"
#filename = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\data_final.csv"
filename = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\data_final_sub.csv"
data = pd.read_csv(filename, sep=',')


In [3]:
'''Saving time in the first part'''
data = data.iloc[0:30, 0:30]
data.shape

(30, 30)

In [4]:
'''Organizing columns names'''
lr = data.columns[[2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]]
y = data.columns[3]
remove = data.columns[[0,1]]
data_complete = data.copy()

In [5]:
'''
Class to work with model parameters
I thought about using the default values as chain starting values, 
however, i encouter problems to change the size of arrays and matrices 
according with my currently k
'''
class parameters:
    def __init__(self, latent_v,latent_sk,latent_pj,latent_phi ,latent_tht, prediction):
        self.ln = latent_v #array with parameters that are only one number [0-c0,1-gamma0,2-eta,3-cj] 
        self.la_sk = latent_sk #array 
        self.la_pj = latent_pj #array
        self.lm_phi = latent_phi #matrix (jk)
        self.lm_tht = latent_tht #matrix  (kv)      
        self.p = prediction #array [intercept, gender, 15 cancer types, k genes]


In [6]:
'''
Declaring my priori values: param.ln = [0-c0,1-gamma0,2-eta,3-cj], param.la_sk, param.la_pj, 
param.lm_phi, param.lm_tht and param.p = [] 
#j=sample of patients 
#v=genes
#k=latent
#UPDATE PARAM 
'''
def priori_l(param):
    k = len(param.la_sk)
    j = len(param.la_pj)
    v = param.lm_phi.shape[0]
    #Prioris on the K plate
    s0 = 1 #define better numbers in the future
    t0 = 1 #define better numbers in the future 
    c0 = gamma.pdf(x=param.ln[0],a = s0,scale = t0) 
    a0 = 1 #define better numbers in the futureT
    b0 = 1 #define better numbers in the future
    gamma0 = gamma.pdf(x=param.ln[1],a = a0,scale = b0)
    sk = gamma.pdf(x=param.la_sk,a=param.ln[1]/k,scale=param.ln[0])#array(k) gamma(3,5): mean is 15
    s0 = 1
    w0 = 1
    eta = gamma.pdf(x=param.ln[2], a = s0, scale = w0) #1
    #print(param.lm_phi.shape,np.repeat(param.ln[2],v).shape)
    phivk = dirichlet.pdf(x=param.lm_phi, alpha=np.repeat(param.ln[2],v)) #MATRIX #1
    #print(phivk.shape)
    priorik = np.log(c0)+np.log(gamma0)+np.log(sk).sum()+np.log(eta)+np.log(phivk).sum()
    #priories on the J plate
    a0 = 1 #define better numbers in the future
    b0 = 1 #define better numbers in the future
    pj = beta.pdf(x=param.la_pj,a=a0,b=b0) #same numbers? #array
    e0 = 2
    f0 = 9
    cj = gamma.pdf(param.ln[3], a = e0, scale = f0)
    priorij = np.log(pj).sum()+np.log(cj) #array    
    #prioris on K and J plate
    thetakj = gamma.pdf(x=np.transpose(param.lm_tht),a=param.la_sk.reshape(1,k),scale = param.ln[3]) #MATRIX 
    priorikj = np.log(thetakj).sum()
    return (priorik+priorij+priorikj)
  
#increase the sd to decrease the .sum()//increase uncertainity about parameters    
def priori_p(param):
    '''
    #add prioris of logistic regression 
    #features are the theta_kj parameters and clinical info
    #one parameter for each latent variable + one parameter for each clinical features
    '''
    k = len(param.la_sk)
    mean = [-len(param.p),1] #intercept and gender
    sd = [1,0.5]
    mean.extend(np.repeat(1/15, 15))
    sd.extend(np.repeat(0.5/15,15))
    mean.extend(np.repeat(1,k))
    sd.extend(np.repeat(0.05,k))
    lr = norm.pdf(param.p,loc = mean, scale = sd) #loc/mean, scale/sd
    return (np.log(lr).sum()) 

In [7]:
'''
Declare all the likelihood distributions in two separete functions
#_F represents the factor model likelihood and _P the prediction model likelihood
'''

#PASS DATA ONLY WITH RELEVANT COLUMNS FOR THIS PROBLEM 
def likelihood_F(param,dataf):
    #print(dataf.head())
    '''phi(jk) x theta(kv) must be matrix , final dim is j patients x v genes '''
    #phi_theta = np.matmul(param.lm_phi, param.lm_tht) 
    #phi_theta = np.transpose(np.matmul(param.lm_phi, param.lm_tht)) 
    phi_theta = np.dot(param.lm_phi, param.lm_tht)
    phi_theta = np.transpose(phi_theta)
    #data_F = data.drop(lr,axis = 1)
    #data_F = data_F.drop(y,axis = 1)
    #print('lf: lm_phi ', start.lm_phi.shape, ' lm_tht shape',start.lm_tht.shape )
    la_pj = np.repeat(start.la_pj, dataf.shape[1]).reshape(start.la_pj.shape[0],dataf.shape[1])
    nvj = nbinom.pmf(dataf,n = phi_theta,p= la_pj) #matrix and array?
    nvj += 0.000001  #LOOK FOR BETTER SOLUTION 
    return np.log(nvj).sum()

#PASS DATA ONLY WITH RELEVANT COLUMNS FOR THIS PROBLEM 
#HERE (2854,) 100

def likelihood_P(param, datap,y):
    '''[intercept, gender, 15 cancer types, k genes]'''
    xw = param.p[0]
    #print(datap.head())
    #print('xw: ', xw[0:10],'datap.shape[1]: ',datap.shape[1], 'param.p: ',param.p.shape)
    for i in np.arange(1,datap.shape[1]):
        xw = xw+param.p[i]*datap[datap.columns[i-1]]
    
    #theta = t(param.lm_tht)
    #print('param.lm_tht shape',param.lm_tht.shape)
    theta = np.transpose(param.lm_tht)
    aux = datap.shape[1]
    #print('xw shape(ok)',xw.shape, 'theta shape - wrong ',theta.shape, 'datap shape',datap.shape, 'param p shape',param.p.shape )
    #xw shape (2854,) theta shape (1981, 100) datap shape (2854, 16) param p shape (103,)

    for j in np.arange(0,theta.shape[1]):
        xw = xw + param.p[aux+j]*theta[:,j]
    
    xwy = 0
    for k in np.arange(1,len(y)): 
        xwy = xwy+xw[k]*y[k]
    
    lxp = sum(-np.log(1+np.exp(xw)))
    #page 2 http://www.medicine.mcgill.ca/epidemiology/joseph/courses/EPIB-621/bayeslogit.pdf
    #http://www.utstat.utoronto.ca/reid/sta2201s/2014/feb14-annotated-copy.pdf
    #https://www.statlect.com/fundamentals-of-statistics/logistic-model-maximum-likelihood
    return xwy + lxp

In [8]:
'''
Definition of the posterior distribution and the split of the datasets
Note: the parameters are ok 
'''
def posterior(param,data_F,data_P,y,k):
    j = data.shape[0]
    #print(data_F.head())
    lf = likelihood_F(param,data_F)
    lp = likelihood_P(param, data_P,y)
    pl = priori_l(param)
    pp = priori_p(param)
    #print(lf,lp,pl,pp)
    return (lf+lp+pl+pp)

In [9]:
'''
Proposal distribution
'''

#Proposal values for the parameters related to the factor model 
#Repete the parameters related to prediction and only propose
#new values for the factor analysys part
#output is the parameters class 
def proposal_f(current):
    new = parameters(np.random.normal(current.ln,0.005),
                     np.random.normal(current.la_sk,0.005),
                     np.random.normal(current.la_pj,0.005),
                     np.random.normal(current.lm_phi,0.005),
                     np.random.normal(current.lm_tht,0.05), 
                     current.p)
    new.lm_phi[new.lm_phi<0] = 0.00001
    col_sums = new.lm_phi.sum(axis=0)
    new.lm_phi = new.lm_phi / col_sums[np.newaxis,:]
    return new

#Proposal values for the parameters related to logistic regression 
#Repete the parameters related to factor analysis part and propose
#new values for the logistc regression parameters 
#output is the parameters class 
def proposal_p(current):
    new = parameters(current.ln, current.la_sk, current.la_pj, 
                     current.lm_phi, current.lm_tht, 
                     np.random.normal(current.p,0.05))
    return new

In [22]:
'''
Creatint the MCMC for the model
MCMC(
startvalue = initial value for the parameters
iterations = 
data = complete data with all columns 
k = number of latent variables
remove, lr, y = columns names to be removed, presente only in the logistic regression part and y
)
'''
def MCMC(startvalue, iterations, data,k, lr,y):
    '''Splitting dataset'''
    data_P = data[lr]
    data_F = data.drop(lr,axis = 1)
    data_F = data_F.drop(y,axis = 1)
    y = data[y]
    
    '''
    Initialization of the chains
    Note: chain_f has elements from chain_p and vice-versa. Take care to not use incorrectly
    '''     
    chain_f = []
    chain_p = []
    chain_f.append(startvalue)
    chain_p.append(startvalue)
    #chain1[0]=proposalvalues1(startvalue1)
    #chain2[0]=proposalvalues2(startvalue2)
    
    for i in np.arange(1,iterations):
        '''Factor Analysis - Latent Features'''
        #use chain_f or chain_p don't make difference here because
        #the only parameters changed are the logistic regression and 
        #they aren't used in the factor analysis part. 
        param_new_f = proposal_f(chain_f[i-1])
        param_cur_f = chain_f[i-1] 
        if i%100 == 0: 
            print('iteration ',i)
        prob_f = np.exp(posterior(param_new_f,data_F,data_P,y,k)-posterior(param_cur_f,data_F,data_P,y,k))
        if np.random.uniform(0,1,1)<prob_f:
            chain_f.append(param_new_f)
        else:
            chain_f.append(param_cur_f) 
        '''Logistic Regression - Prediction'''
        #chain_f[i] has the most update latent parameters and haven't changed the 
        #prediction parameters from [i-1] iteration
        param_new_p = proposal_p(chain_f[i-1])
        param_cur_p = chain_f[i-1]
    
        prob_p = np.exp(posterior(param_new_p,data_F,data_P,y,k)-posterior(param_cur_p,data_F,data_P,y,k))
        if np.random.uniform(0,1,1)<prob_p:
            chain_p.append(param_new_p)
        else:
            chain_p.append(param_cur_p)     

    return chain_p, chain_f
        
        

In [11]:
'''Non informative prioris: dirichlet has only 1, gamma distribution with 1 average, etc'''
aux = len(lr)+1
data = data.drop(remove,axis = 1)

start = parameters([3,5,2,1], #ln 
                   np.repeat(15,k), #la_sk
                   np.repeat(0.5,data.shape[0]), #la_pj
                   np.repeat(1/(data.shape[1]-aux),(data.shape[1]-aux)*k).reshape((data.shape[1]-aux),k) ,#lm_phi v x k 
                   np.repeat(1,(data.shape[0])*k).reshape(k,(data.shape[0])), #lm_theta k x j
                   np.concatenate(([-(k+aux)], np.repeat(1,k+aux-1))))  #p, k+aux-1  because intercept is already counted



In [23]:
#data = data.drop(remove,axis=1)
output_p, output_f = MCMC(start,1000,data,k,lr,y)




C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in log


iteration  100
iteration  200
iteration  300
iteration  400
iteration  500
iteration  600
iteration  700
iteration  800
iteration  900


In [28]:
output_f[19].la_pj

array([0.48174978, 0.49325621, 0.52090883, 0.49204359, 0.48788733,
       0.48178097, 0.4902571 , 0.50121692, 0.52385536, 0.50854497,
       0.49948279, 0.49392971, 0.47699266, 0.49075841, 0.51282081,
       0.48331954, 0.45619345, 0.50528673, 0.49021117, 0.52731664,
       0.49553645, 0.50577761, 0.544396  , 0.4889385 , 0.47693047,
       0.5132861 , 0.51786684, 0.51273601, 0.50165593, 0.50773795])

1